In [1]:
import requests
import json
import pandas as pd
from datetime import datetime as dt
from datetime import timezone
from dateutil import parser

# Show all columns when printing a pandas DataFrame
pd.options.display.max_columns = None

In [2]:
# Craetes a list of the fileds with a file as input
def read_fields(filename):
    fields = []
    fd = open(filename)
    lines = fd.readlines()
    for line in lines:
        fields.append("data."+line[:-1])
    fd.close()
    return fields

def remove_dot_data(fields):
    fields_no_data=[]
    for field in fields:
        field_no_data = field.replace("data.","")
        fields_no_data.append(field_no_data)
    return fields_no_data

def read_token(filename):
    fd = open(filename)
    line = fd.readline()
    # Remove the 'newline' character if found
    if line[len(line)-1] == '\n':
        line = line[:-1]
    fd.close()
    return line

In [78]:
# Where to get the authorization token
file_token="./token"

# Where to get the list of fields to retrieve
file_fields="./fields"

# Read the authorization token
token = read_token(file_token)
headers = {'Authorization': 'Bearer '+token,}

# Set the start(min) and end(max) dates to perform the query onto
min_date_human_readable = "Jun 25 00:00:00 UTC 2019"
max_date_human_readable = "Jun 25 23:59:59 UTC 2019"

# Get timestamps in miliseconds
min_date = str(int(parser.parse(min_date_human_readable).timestamp() * 1000))
max_date = str(int(parser.parse(max_date_human_readable).timestamp() * 1000))

# The query on lucene syntaxis
query =               "data.Type:analysis"
query = query + " AND  data.Status:Completed"
query = query + " AND  data.JobUniverse:5"
query = query + " AND  data.CRAB_Workflow:190529_225118*gomber_crab_job_TTJets_FXFX"
#query = query + " AND  data.CRAB_Id:7716"


# Read the list of fields to retrieve from a file
fields = read_fields(file_fields)
fields= ['data.CRAB_Workflow', 'data.CRAB_Id', 'data.CRAB_Retry','data.ScheddName']
fields_no_data = remove_dot_data(fields)

# Number of records to retreive
num_records = 10000

In [79]:
print("Query:")
print(query)
print("Fields no data:")
print(fields_no_data)

data = {
        "size":num_records,
        "query":{
            "bool":{
                "filter":[
                    {"range":{"data.RecordTime":{"gte":min_date,"lte":max_date,"format":"epoch_millis"}}},
                    {"query_string":{
                        "analyze_wildcard":"true",
                        "query":query
                        }
                    }
                ]
            }
        },
        "_source":fields
}

data_string = json.dumps(data)

Query:
data.Type:analysis AND  data.Status:Completed AND  data.JobUniverse:5 AND  data.CRAB_Workflow:190529_225118*gomber_crab_job_TTJets_FXFX
Fields no data:
['CRAB_Workflow', 'CRAB_Id', 'CRAB_Retry', 'ScheddName']


In [80]:
# Send the query (It takes few seconds)
response = requests.get('https://monit-grafana.cern.ch/api/datasources/proxy/9014/_search', headers=headers, data=data_string)

In [81]:
# Get the data from the response
d = response.json()
# Create a pandas DataFrame with the data retreived
clean_records=[]
no_data_count=0
for record in d['hits']['hits']:
    try:
        clean_record= record['_source']['data']
        clean_records.append(clean_record)
    except:
        no_data_count = no_data_count +1
df = pd.DataFrame(clean_records) 

In [83]:
df.sort_values(['CRAB_Id','CRAB_Retry'])

,CRAB_Id,CRAB_Retry,CRAB_Workflow,ScheddName
281,100,7,190529_225118:gomber_crab_job_TTJets_FXFX,crab3@vocms0107.cern.ch
209,1012,7,190529_225118:gomber_crab_job_TTJets_FXFX,crab3@vocms0107.cern.ch
381,1012,8,190529_225118:gomber_crab_job_TTJets_FXFX,crab3@vocms0107.cern.ch
399,1012,9,190529_225118:gomber_crab_job_TTJets_FXFX,crab3@vocms0107.cern.ch
12,1024,3,190529_225118:gomber_crab_job_TTJets_FXFX,crab3@vocms0107.cern.ch
171,1026,3,190529_225118:gomber_crab_job_TTJets_FXFX,crab3@vocms0107.cern.ch
349,1080,7,190529_225118:gomber_crab_job_TTJets_FXFX,crab3@vocms0107.cern.ch
426,1081,6,190529_225118:gomber_crab_job_TTJets_FXFX,crab3@vocms0107.cern.ch
470,1081,7,190529_225118:gomber_crab_job_TTJets_FXFX,crab3@vocms0107.cern.ch
319,1081,8,190529_225118:gomber_crab_job_TTJets_FXFX,crab3@vocms0107.cern.ch


In [32]:
# Print the DataFrame
df['Exit'] = df['Chirp_CRAB3_Job_ExitCode'].apply(lambda x: 0 if x == 0.0 else 1)

In [52]:
df_g = df.groupby(['CRAB_Workflow','CRAB_Id']).agg({"CRAB_Retry":"count", "Exit":"sum"})

In [53]:
df_g

CRAB_Retry  Exit
CRAB_Workflow                                      CRAB_Id                  
190527_220527:dwinterb_crab_VBFHToTauTau_M125_1... 6332              1     0
                                                   8083              1     0
                                                   8246              1     0
                                                   8313              1     1
                                                   8569              1     1
                                                   8687              1     0
190604_044933:mdjordje_crab_test_SingleMuon_Run... 1912              1     1
190604_174157:dwinterb_crab_VBFHToTauTau_M125_1... 1110              1     0
                                                   1117              1     0
                                                   3473              1     0
                                                   3487              1     0
                                                   3491              1     0
190604_191627:mdjordje_crab_test_SingleMuon_Run... 2719              1     0
                                                   2880              1     0
                                                   2881              1     0
                                                   3022              1     1
190605_221327:spigazzi_crab_legacyRun2FullV1_Do... 915               1     0
190605_221949:spigazzi_crab_legacyRun2FullV1_GJ... 23                1     0
190606_055702:spigazzi_crab_legacyRun2FullV1_Si... 133               1     1
190606_060524:spigazzi_crab_legacyRun2FullV1_Si... 1006              1     0
                                                   1015              1     0
                                                   1036              1     0
                                                   1040              1     0
                                                   1045              2     0
                                                   1068              1     0
                                                   1080              1     0
                                                   1097              1     0
                                                   1107              1     0
                                                   1121              1     0
                                                   1134              1     0
...                                                                ...   ...
190626_053123:sciaba_crab_HC-140-AnySite-76600-... 17                1     0
                                                   18                1     0
                                                   45                1     0
                                                   65                1     0
                                                   81                1     0
                                                   97                1     0
190626_053434:sciaba_crab_HC-159-AnySite-76601-... 102               1     0
                                                   117               1     1
                                                   140               1     0
                                                   142               1     0
                                                   15                1     0
                                                   158               1     0
                                                   177               1     0
                                                   180               1     0
                                                   183               1     0
                                                   21                1     0
                                                   26                1     0
                                                   43                1     0
                                                   46                1     0
                                                   57   

In [54]:
df_g['diff'] = df_g['CRAB_Retry'] - df_g['Exit']

In [55]:
df_g[df_g['diff']>1].count()

CRAB_Retry    49
Exit          49
diff          49
dtype: int64

In [56]:
df_g.count()

CRAB_Retry    9315
Exit          9315
diff          9315
dtype: int64